In [2]:
import warnings
#warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from datetime import datetime

from sklearn import model_selection
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

from datacleaner import autoclean


from xgboost import XGBClassifier, XGBRegressor

In [40]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

test_df = test_df.drop(['Income'],axis=1)

val_instances = test_df['Instance']


Y = train_df['Income in EUR']
X_raw = train_df.drop(['Income in EUR'], axis=1)


# This is kinda ugly but w/e... Load in X_test here too, and combine into one massive X dataset.
# Then can later separate for training. Helps labels be generated correctly and all that.

X_raw = pd.concat([X_raw, test_df],axis='rows').reset_index()

# Drop instance labels.
X_raw = X_raw.drop(['Instance'], axis=1)


# Fill in null years & ages
X_raw['Year of Record'] = X_raw['Year of Record'].fillna(2000)
#X_raw['Age'] = X_raw['Age'].fillna(20)

print(X_raw)
print(X_raw.shape)

        index  Year of Record   Gender   Age                   Country  \
0           0          1997.0        0  41.0                   Belarus   
1           1          1996.0    other  41.0                 Singapore   
2           2          2018.0    other  28.0                    Norway   
3           3          2006.0    other  33.0                      Cuba   
4           4          2010.0   female  46.0      United Arab Emirates   
5           5          1991.0     male  60.0                   Liberia   
6           6          1982.0      NaN  71.0        State of Palestine   
7           7          2008.0     male  36.0                    Israel   
8           8          2015.0     male  43.0               South Sudan   
9           9          2019.0   female  16.0                      Cuba   
10         10          1981.0     male  51.0                Kyrgyzstan   
11         11          1989.0    other  15.0                      Togo   
12         12          1997.0      NaN

In [3]:
# Find average earning and city size for each profession
# Create df
blah = pd.concat([train_df['Country'], train_df['Size of City'], Y], axis=1)
b = blah.groupby('Country').mean().reset_index()

print(b)

                      Country  Size of City  Income in EUR
0                 Afghanistan  6.373037e+05   8.879485e+05
1                     Albania  8.724576e+05   3.093651e+05
2                     Algeria  9.825891e+05   5.836315e+05
3                      Angola  1.190360e+06   6.436873e+05
4                   Argentina  1.019613e+06   1.104589e+06
5                     Armenia  7.791162e+05   2.706201e+05
6                   Australia  7.919086e+05   4.447110e+05
7                     Austria  8.103452e+05   2.881055e+04
8                  Azerbaijan  7.866670e+05   5.198797e+04
9                     Bahamas  3.960233e+05   9.802125e+05
10                    Bahrain  7.441467e+05   4.918415e+05
11                   Barbados  1.213216e+06   9.864700e+05
12                    Belarus  8.046710e+05   3.311681e+04
13                    Belgium  8.311698e+05   1.250704e+05
14                     Belize  1.276542e+06   9.253659e+05
15                      Benin  8.118822e+05   1.476909e+

In [7]:
# Country GDP Data
# from iso3166 import countries
import pycountry


# # Create map from countries in dataset to country code.
df_countries = X_raw['Country']
keys = df_countries.unique()
print(keys)

countrycode = {}
for k in keys:
    try:
        c = pycountry.countries.lookup(k)
        countrycode[k] = c.alpha_3
        
    except LookupError:
        #print("couldn't find", k)
        # All just syria cause w/e
        countrycode[k] = 'SYR'
        
  
# df_gdp = pd.read_csv('GDP_Data.csv')
# df_gdp = df_gdp.loc[df_gdp['MEASURE'] == 'MLN_USD']

# cGood = 0
# cBad = 0

# year_ave = {}
# for y in range(1980, 2019):
#     ave = df_gdp.loc[df_gdp['TIME'] == y]['Value'].mean()
#     year_ave[y] = ave
#     #print(y, ave)
    
# year_ave[2019] = year_ave[2018]

# def lookup(row):
#     global cGood
#     global cBad
    
#     if cGood % 100 == 0:
#         print(cGood, cBad)
    
#     #print(row)
    
#     year = row['Year of Record']
#     code = countrycode[row['Country']]
    
#     try:
#         #v = df_gdp[('LOCATION' == code) & ('TIME' == year) & ('MEASURE' == 'MLN_USD')]
#         v = df_gdp[(df_gdp['LOCATION'] == code) & (df_gdp['TIME'] == year)].iloc[0]['Value']

#         #print(v)
#         cGood += 1
#         return v
#     except IndexError:
#         #print("Couldn't find..", code, year)
#         cBad = cBad + 1
#         # If no info, just give year average...
#         # Try returning a fraction of it, since the countries not included in the list
#         # are more likely to be poorer.
#         return year_ave[2000]



# gdp = X_raw.apply(lambda row: lookup(row), axis=1)
# print(gdp)

['Belarus' 'Singapore' 'Norway' 'Cuba' 'United Arab Emirates' 'Liberia'
 'State of Palestine' 'Israel' 'South Sudan' 'Kyrgyzstan' 'Togo' 'Finland'
 'Sierra Leone' 'Papua New Guinea' 'Czechia' 'Paraguay' 'Belgium'
 'Costa Rica' 'Senegal' 'Congo' 'Slovakia' 'Burundi' 'Portugal' 'Tunisia'
 'Lebanon' 'Azerbaijan' 'Chile' 'Botswana' 'Jordan' 'Zimbabwe' 'Rwanda'
 'Greece' 'Turkmenistan' 'Sweden' 'El Salvador' 'Somalia' 'Haiti'
 'Tajikistan' 'Hungary' 'Peru' 'Laos' 'Austria' 'Bolivia' 'Kazakhstan'
 'Nicaragua' 'Moldova' 'Gabon' 'Bulgaria' 'Namibia' 'Croatia'
 'Central African Republic' 'Ghana' 'Denmark' 'Cambodia' 'Netherlands'
 'Niger' 'Mozambique' 'Ireland' 'Mongolia' 'Luxembourg' 'Benin' 'Kuwait'
 'Serbia' 'Chad' 'North Korea' 'Dominican Republic' 'Burkina Faso'
 'Switzerland' 'Ecuador' 'Honduras' 'Libya' 'Syria' 'Uruguay' 'Qatar'
 'Mauritania' 'Jamaica' 'Guinea-Bissau' 'Panama' 'Cameroon'
 'Bosnia and Herzegovina' 'North Macedonia' 'Oman' 'Eritrea' 'Australia'
 'Guinea' 'Saudi Arabia' 'Al

In [20]:
    
df_gpi = pd.read_csv("API_NY.GNP.PCAP.CD_DS2_en_csv_v2_248110.csv")
c = 0


def lookupGPI(row):
    global c
    if c % 1000 == 0:
        print(c)
    c += 1
    
    year = row['Year of Record']
    if year == np.nan:
        year = 2000
    if year > 2018:
        year = 2018
    code = countrycode[row['Country']]
    
    #try:
    v = df_gpi[(df_gpi['Country Code'] == code)].iloc[0][str(int(year))]
    #print(year, code, v)
    if v == np.nan:
        v = 200
    return v
    
    
gpi = X_raw.apply(lambda row: lookupGPI(row), axis=1)
gpi.to_csv('gpi.csv', index=False)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


/home/oisincar/.local/lib/python3.6/site-packages/ipykernel_launcher.py:27: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


In [95]:
# Preprocess... Assemble our X - input to training.
X = pd.DataFrame()

gpi = pd.read_csv('./gpi.csv', header=None, names=['GPI'])


year = X_raw['Year of Record']
# Normalize
#X['Year'] = year #(year-year.min())/(year.max() - year.min())
X['Year'] = year

# Gender... 
gender = X_raw['Gender']
print("gen", gender.shape)
gender = gender.replace([0, '0', np.nan], "unknown")
dummy_gender = (pd.get_dummies(gender))
X = pd.concat([X, dummy_gender], axis=1)

# Country
#country_dum = pd.get_dummies(X_raw['Country']) #159 countries
#X = pd.concat([X, country_dum], axis=1)


country = X_raw['Country']#.str.lower()
# Add so we can merge on it in a sec
X = pd.concat([X, country], axis=1)
#X['Country'] = country


# GPI...
a = autoclean(gpi)
print(a['GPI'])
X['GPI'] = a['GPI']
train_df['GPI'] = a['GPI'][0:111993]
print("x", X.shape)




#print(train_df['Profession'].str.lower(), train_df['Size of City'], Y)
# Create averages only from training set.
countryAves = pd.concat([train_df['Country'],#.str.lower(),
                        train_df['Size of City'],
                        train_df['Age'],
                        train_df['GPI'],
                        Y], axis=1)

b = countryAves.groupby('Country').mean().reset_index()
b = b.rename({ 'Size of City':'CountryAveCitySize',
               'Income in EUR':'CountryAveEarning',
               'GPI':'CountryAveGPI',
               'Age':'CountryAveAge'}, axis=1)

print(b)
# Empty spaces can be added in later by autoclean...

#X = pd.merge(X, b, on='Profession', how='left', sort=False)
X = X.reset_index().merge(b, on='Country', how="left").set_index('index')



# Used to grab country info in a sec
tmp_train_df = train_df.reset_index().merge(b, on='Country', how="left").set_index('index')
#print(tmp_train_df)











# Yearly stuff
e = pd.concat([train_df['Country'],
               train_df['Year of Record'],
               Y], axis=1)

e = e.groupby(['Country', 'Year of Record']).mean().reset_index()
e = e.rename({ 'Income in EUR':'CountryYearlyAveEarning',
                'Year of Record':'Year'}, axis=1)
print("yearly stuff!!! -----------------------------------------")
print(e)
X = X.reset_index().merge(e, on=['Country', 'Year'], how="left").set_index('index')













X = X.drop('Country', axis=1)

# Age
X['Age'] = X_raw['Age']
X['Agelog'] = np.log(X_raw['Age'])


# Professions
# prof = X_raw['Profession'].str.lower()
# profession_dum = pd.get_dummies(prof) # 1339

# # Drop rare professions
# vc = prof.value_counts()
# rare_prof = vc[vc <= 5].index.tolist()
# print(rare_prof)
# profession_dum = profession_dum.drop(rare_prof, axis=1)
# X = pd.concat([X, profession_dum], axis=1)

X['CityPop'] = X_raw['Size of City']
#X['CityPoplog'] = np.log(X_raw['Size of City'])

prof = X_raw['Profession'].str.lower()
# Add so we can merge on it in a sec
X = pd.concat([X, prof], axis=1)


#print(train_df['Profession'].str.lower(), train_df['Size of City'], Y)
# Create averages only from training set.
professionAves = pd.concat([train_df['Profession'].str.lower(),
                            train_df['Size of City'],
                            train_df['Age'],
                            tmp_train_df['CountryAveCitySize'],
                            tmp_train_df['CountryAveEarning'],
                            tmp_train_df['CountryAveAge'],
                            Y], axis=1)

b = professionAves.groupby('Profession').mean().reset_index()
b = b.rename({ 'Size of City':'ProfessionAveCitySize',
               'Income in EUR':'ProfessionAveEarning',
               'Age':'ProfessionAveAge',
             'CountryAveAge':'CountryProfAveAge',
             'CountryAveEarning':'CountryProfAveEarning',
             'CountryAveCitySize':'CountryProfAveCitySize'}, axis=1)


earningAves = pd.concat([train_df['Profession'].str.lower(), Y], axis=1)

c = earningAves.groupby('Profession').quantile(0.25).reset_index()
c = c.rename({'Income in EUR':'Income25'}, axis=1)
print(c)
d = earningAves.groupby('Profession').quantile(0.75).reset_index()
d = d.rename({'Income in EUR':'Income75'}, axis=1)


# Empty spaces can be added in later by autoclean...
#X = pd.merge(X, b, on='Profession', how='left', sort=False)
X = X.reset_index().merge(b, on='Profession', how="left").set_index('index')
X = X.reset_index().merge(c, on='Profession', how="left").set_index('index')
X = X.reset_index().merge(d, on='Profession', how="left").set_index('index')


X = X.drop('Profession', axis=1)




# Extra bits... 
X['ayy'] = X['ProfessionAveEarning'] / X['CountryAveEarning']
X['ayyinc'] = X['Income25'] / X['ProfessionAveEarning']
X['ayyinc2'] = X['Income75'] / X['ProfessionAveEarning']

X['ayyinc3'] = X['CountryYearlyAveEarning'] / X['CountryAveEarning']

X['ayy2'] = X['Age'] / X['ProfessionAveAge']
X['ayy3'] = X['Age'] / X['CountryAveAge']
X['ayy4'] = X['CityPop'] / X['CountryAveCitySize']

#X['RelativeGPI'] = X['GPI'] / X['CountryAveGPI']
#print(X['RelativeGPI'])

X = X.drop('CityPop', axis=1)
X = X.drop('CountryAveCitySize', axis=1)
X = X.drop('CountryProfAveAge', axis=1)
X = X.drop('CountryAveAge', axis=1)

X = X.drop('Income25', axis=1)
X = X.drop('Income75', axis=1)

X = X.drop('GPI', axis=1)
X = X.drop('CountryAveGPI', axis=1)
# Maybe?
#X = X.drop('CountryAveEarning', axis=1)



# Adding parameter for 'is assistant', engineer, scientist etc.
# X['isassistant'] = 0
# X['isassistant'].loc[prof.str.contains("assistant") == True] = 1

# X['isengineer'] = 0
# X['isengineer'].loc[prof.str.contains("engineer") == True] = 1


# Degrees
deg = X_raw['University Degree']
deg = deg.replace([0, '0', 'No'], 'NoDegree')
degree_dum = pd.get_dummies(deg) # 4 degrees
print(X.shape, degree_dum.shape)
X = pd.concat([X, degree_dum], axis=1)

# HairCol
#haircol_dum = pd.get_dummies(X_raw['Hair Color'])
#X = pd.concat([X, haircol_dum], axis=1)

X['Height'] = X_raw['Body Height [cm]']
#X['Heightsq'] = np.power(X['Height'], 2)
X['Heightlg'] = np.log(X['Height'])


#X['YesGlasses'] = X_raw['Wears Glasses']
#X['NoGlasses'] = 1-X_raw['Wears Glasses']

# Just bulletproof by adding adding in default vals.
#X['GDP'] = gdp#.fillna(4000)
#X['GPI'] = gpi#.fillna(200)

# Just fill in all other nans with 0 cause .. o well.
#X.fillna(0)
print("Cleaning...")
X = autoclean(X)
print(X)



print("Saving...")
print("Final shape", X.shape)
pd.DataFrame(X).to_csv('X.csv', index=False)
print(Y.shape)
Y.to_csv('y.csv', index=False)

gen (185223,)
0           1530.0
1          25620.0
2          80790.0
3           4530.0
4          33390.0
5           2590.0
6           1750.0
7          26080.0
8           1020.0
9           2590.0
10          2590.0
11           400.0
12         26650.0
13           200.0
14           470.0
15          2590.0
16          5680.0
17         27120.0
18         11700.0
19          3170.0
20           960.0
21          1050.0
22         14050.0
23          2590.0
24         17200.0
25           230.0
26         17240.0
27          5780.0
28          2530.0
29          1290.0
            ...   
185193      1950.0
185194       420.0
185195      1330.0
185196      2590.0
185197      2590.0
185198     19280.0
185199     17510.0
185200      5520.0
185201     52520.0
185202     18880.0
185203       940.0
185204      1310.0
185205      5690.0
185206      2590.0
185207     39700.0
185208    104340.0
185209     69980.0
185210     16870.0
185211     15110.0
185212       570.0
185213     11700.

0.25                                         Profession       Income25
0                                        .net developer   29083.175930
1                               .net software developer   61133.102830
2                    accessibility outreach coordinator   17204.790070
3                         accessibility program manager   35186.337975
4                                    account executive   644524.183000
5                                       account manager   18152.213110
6                                   accountable manager   47714.929890
7                           accountable project manager  133089.917825
8                                            accountant   45913.048555
9                         accounting investment analyst   32654.165710
10                                   accounts assistant   38770.278350
11                                       accounts clerk   43959.068725
12                                     accounts manager   42031.582202
13    

(111993,)


/home/oisincar/.local/lib/python3.6/site-packages/ipykernel_launcher.py:238: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


In [7]:
X = pd.read_csv('X.csv')
y = pd.read_csv('y.csv', header=None)

In [96]:
# xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',
#                     silent=True, nthread=1)

xgb = XGBRegressor(colsample_bytree=0.4,
                 gamma=0,                 
                 learning_rate=0.07,
                 max_depth=3,
                 min_child_weight=1.5,
                 n_estimators=10000,       # Was: 10000                                                              
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42)

xgb = XGBRegressor(colsample_bytree=0.4,
                   eval_metric='rmse',
                 gamma=0,
                   nthread=16,
                   verbosity=1,
                 learning_rate=0.1,
                 max_depth=5,             # Was: 3
                 min_child_weight=1,
                 n_estimators=1000,       # Was: 10000                                                              
                 #reg_alpha=0.75,
                 #reg_lambda=0.45,
                 #subsample=0.6,
                 seed=42)

# Best so far...
# xgb = XGBRegressor(colsample_bytree=0.4,
#                    eval_metric='rmse',
#                  gamma=0,
#                    nthread=8,
#                    verbosity=2,
#                  learning_rate=0.1,
#                  max_depth=5, 
#                  min_child_weight=1,
#                  n_estimators=1000,                                                     
#                  #reg_alpha=0.75,
#                  #reg_lambda=0.45,
#                  #subsample=0.6,
#                  seed=42)

xgb = XGBRegressor(
    colsample_bytree= 0.8,
    learning_rate=0.1,
    gamma=0.3,
    nthread=8,
    max_depth=4, 
    min_child_weight=2, 
    n_estimators=1000,
    subsample=0.8)

# xgb = XGBRegressor(colsample_bytree=0.4,  # 77810 to beat
#                    eval_metric='rmse',
#                  gamma=0,
#                    nthread=8,
#                    verbosity=2,
#                  learning_rate=0.1,
#                  max_depth=5,
#                  min_child_weight=1,
#                  n_estimators=1000,       # Was: 10000                                                              
#                  reg_alpha=0.75,
#                  reg_lambda=0.45,
#                  subsample=0.6,
#                  seed=42)

In [97]:

print("Startin'!")

X_partial = X.iloc[0:111993,:] 
X_val     = X.iloc[111993:,:]

X_train, X_test, y_train, y_test = train_test_split(X_partial, Y, test_size=0.01, random_state=42)

# Clip y to not be so large, reducing outliers.
#y_train = Y #Y.clip(0,1000000)

print("he", xgb.fit(X_train, y_train))
print("Done")

Startin'!
[00:57:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/home/oisincar/.local/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


he XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, gamma=0.3,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=4, min_child_weight=2, missing=None, n_estimators=1000,
             n_jobs=1, nthread=8, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=0.8, verbosity=1)
Done


In [99]:
y_train_pred = xgb.predict(X_train)
RMSE = np.sqrt(mean_squared_error(y_train, y_train_pred))

y_test_pred = xgb.predict(X_test)
RMSE2 = np.sqrt(mean_squared_error(y_test, y_test_pred))

print(RMSE.round(4), RMSE2.round(4))

39755.8818 62081.4482


In [11]:
skf = KFold(n_splits=folds, shuffle = True, random_state = 1001)

a = model_selection.cross_val_score(xgb, X, Y, cv=skf, scoring='neg_mean_squared_error')
print([np.sqrt(-v) for v in a])

NameError: name 'folds' is not defined

In [ ]:
#print(X['GPI'])


In [100]:
# Make predictions & SAVE!!!!!!!!!!!
y_val = xgb.predict(X_val)

submission = pd.DataFrame({
        "Instance": val_instances,
        "Income": y_val.flatten()
    })
submission = pd.DataFrame.from_records(submission)
submission.to_csv('submission.csv', index=False)


In [66]:
# PARAMETER SEARCH!!!
params = {'min_child_weight':[2],
          'learning_rate':[0.1],
          'gamma':[i/10.0 for i in range(3,6)],
          'subsample':[0.8],
          'colsample_bytree':[0.8],
          'n_estimators': [1000],
          'max_depth': [4]}
# Best: 69059.86481430089
# {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 8, 'min_child_weight': 1, 'n_estimators': 1000, 'subsample': 0.8}

# params = {
#         'min_child_weight': [5],
#         'gamma': [1],
#         'subsample': [0.8],
#         'colsample_bytree': [0.8],
#         'max_depth': [5],
#         'n_estimators': [1000],
#         }

# Initialize XGB and GridSearch
xgb = XGBRegressor(nthread=1) # TODO put back to  4/ 10

grid = GridSearchCV(xgb, params, n_jobs=8,
                    scoring='neg_mean_squared_error',
                    cv=3,
                    verbose=2,
                    refit=True
                    )
grid.fit(X_train, y_train)

best_result = grid.best_score_
print(best_result)
print(np.sqrt(-best_result))
print()

best_parameters = grid.best_params_
print(best_parameters)

# # Print the r2 score
#print("Error on test set...")
#y_test_pred = xgb.predict(X_test)
#RMSE = np.sqrt(mean_squared_error(y_test, y_test_pred))

Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   4 out of   9 | elapsed:  2.6min remaining:  3.3min
[Parallel(n_jobs=8)]: Done   9 out of   9 | elapsed:  3.9min remaining:    0.0s
[Parallel(n_jobs=8)]: Done   9 out of   9 | elapsed:  3.9min finished
/home/oisincar/.local/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[00:33:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
-3039376256.488412
55130.538329390656

{'colsample_bytree': 0.8, 'gamma': 0.3, 'learning_rate': 0.1, 'max_depth': 4, 'min_child_weight': 2, 'n_estimators': 1000, 'subsample': 0.8}
